# Neural Networks - Back Propogation

- <font color="red">The following section is, I think, the most complicated thing till now, so</font>
- <font color="red">We take a second to explain the general idea of what we're going to do:</font>

We've already described __*<font color="blue" size="3em">forward propagation</font>*__
- This is the algorithm which takes your neural network and the initial input into that network 
  and <font color="blue">pushes the input through the network</font>
  - It leads to the generation of an output hypothesis, which may be a single real number, but can also be a vector

We're now going to describe __*<font color="blue" size="3em">back propagation</font>*__
- Back propagation basically takes the output you got from your network, 
  <br>compares it to the expected real value $(y)$ and calculates how wrong the network was 
  <br>(i.e. how wrong the parameters were)
- It then, using the error you've just calculated, 
  <font color="blue">back-calculates the error associated with each unit from the preceding layer 
  (i.e. layer L - 1)</font>
- This goes on until you reach the input layer (where obviously there is no error, as the activation is the input)
- These "error" measurements for each unit can be then used to calculate the __*partial derivatives*__
  - <font color="blue">Partial derivatives are then used by *gradient descent* to minimize the cost function</font>
- We use the partial derivatives with gradient descent to try minimize the cost function 
  and update all the $Ɵ$ values
- This repeats until gradient descent reports convergence

<u>A few things to remember</u>:
- There is a <font color="blue">Ɵ matrix for each layer in the network</font>
  - This has each node in layer $l$ as one dimension (rows) and each node in $l+1$ as the other dimension (columns)
- Similarly, there is going to be a <font color="blue">Δ matrix for each layer</font>
  - This has each node as one dimension and each training data example as the other

## Gradient Comutation

- <font color="blue">*Back propagation* is an algorithm used for calculating partial derivatives!</font>
  <br>later used by *gradient descent* for minimizing the cost function


- Neural network cost function:
<img src="images/neural-network-cost-function.png">

- ### <font color="blue">We want to find parameters $Ɵ$ which minimize $J(Ɵ)$</font>

- To do so we can use one of
  - <font color="blue">Gradient descent</font>
  - Advanced optimization algorithms (e.g., <font color="blue">Conjugate gradient, BFGS, L-BFGS</font>)


- To minimize a cost function we just <font color="blue">write code which computes</font> the following
  - <font color="blue">$J(Ɵ)$</font>
    - i.e. the cost function itself! (use the formula above to calculate this value)
  - <font color="blue">Partial derivative terms</font>
    - $Ɵ$ is indexed in 3-dimensions because each layer has a $Ɵ$ matrix associated with it!
      - We want to calculate the partial derivative $Ɵ$ with respect to a single parameter
      <img src="images/D-J-theta.png">
      - The partial derivative term we calculate above is a REAL number (not a vector or a matrix)

- ### <font color="blue">We next focus on how to compute these partial derivative terms</font>
  - We start by talking about the case where we have only one training example
  - The first thing we do is applying <font color="blue">forward propogation</font> to get $H_Ɵ(x)$
  <img src="images/NN-4-layers-Forward-Propogation.png">
  - The forward propagation algorithm operates as follows (vectorized implementation)
    - **Layer 1** (Input)
      - $a^1 = x$
    - **Layer 2**
      - $z^2 = Ɵ^1a^1$    {add $a_0^1$}
      - $a^2 = g(z^2)$
    - **Layer 3**
      - $z^3 = Ɵ^2a^2$    {add $a_0^2$}
      - $a^3 = g(z^3)$
    - **Layer 4** (Output)
      - $z^4 = Ɵ^3a^3$    {add $a_0^3$}
      - $a^4 = H_Ɵ(x) = g(z^4)$

### Next, in order to compute the derivatives, we're going to use
## <font color="blue">Backpropogation Algorithm</font>

- For each node at layer $l$ - calculate <font color="blue">($δ_j^l$)</font> - 
  this is the <font color="blue">error of node $j$ in layer $l$</font>
  - $δ_j^l$ captures the node activation ($a_j^l$) error compared to the "real" value
  - We start from the end: <font color="blue">$δ_j^4 = a_j^4 - y_j$</font>
  - Instead of focussing on each node, let's think about this as a vectorized problem
    - <font color="blue">$δ^4 = a^4 - y$</font>
      - So here $δ^4$ is the vector of errors for the 4<sup>th</sup> layer
      - $a^4$ is the vector of activation values for the 4<sup>th</sup> layer
  - With δ4 calculated, we can determine the error terms for the other layers as follows:
    <img src="images/delta-calc-layers-3-and-2.png">
    - Taking a second to break this down
      - $Ɵ^3$ is the vector of parameters for the 3 -> 4 layer mapping
      - $δ^4$ is (as calculated) the error vector for the 4<sup>th</sup> layer
      - $g'(z^3)$ is the derivative of the activation function $g$ 
        evaluated by the input values given by $z^3$
        - When you calculate this derivative you get $g'(z^3) = a^3$ .\* $(1 - a^3)$
      - So, <font color="blue">$δ^3 = (Ɵ^3)^T δ^4$ .\* $(a^3$ .\* $(1 - a^3))$</font>
        - <font color="red"><b>.\*</b></font> is the element wise multiplication between the two vectors
        - Why element wise? Because this is essentially an extension of individual values in a 
          vectorized implementation
  - There's no $δ^1$ term because that's the input!
 
### Why do we do all this?
 - We do all this to get all the δ terms, and we want the δ terms because through a very complicated derivation 
   <br>you can use δ to get the partial derivative of Ɵ with respect to individual parameters 
   <br>(if you ignore regularization, or regularization is 0, which we deal with later)
   <img src="images/D-J-theta-and-delta.png">

### Putting it all together to get the partial derivatives!
- Training set of m examples
  <img src="images/training-set.png">
- Init the delta values
  - Will be used as accumulators for computing the partial derivatives
  <img src="images/delta-i-j-init.png">
- Loop through the training set:
  - i.e. for each example in the training set - dealing with each example as $(x,y)$
  - Set $a^1$ (activation of input layer) = $x_i$
  - for $i = 1$ to $m$
    - Perform <font color="blue">forward propagation</font> to compute $a^l$ for each layer (l = 1,2, ... L)
    - Then, use the output label for the specific example we're looking at to calculate 
      $δ^L$ where $δ^L = a^L - y^i$
      - So we initially calculate the delta value for the output layer
      - Then, using <font color="blue">back propagation</font> we move back through the network 
        from layer $L-1$ down to layer 1
      - Finally, use Δ to accumulate the partial derivative terms
        <img src="images/delta-i-j-calc.png">
      - You can vectorize the Δ expression too, as
        <img src="images/delta-i-j-calc-vectorized.png">
  - Finally, after executing the body of the loop, exit the for loop and compute
    - (when $j = 0$ we have no regularization term)
    <img src="images/D-i-j-calc.png">
- At the end of ALL this
  - You've calculated all the D terms above using Δ
    - each D term above is a real number!
  - It can be shown that each D is equal to the following
  <img src="images/bp-final-step.png">

### Finally, we have calculated the partial derivative for each parameter
- We can then use these in gradient descent or one of the advanced optimization algorithms

## Back propagation intuition


### Forward propagation with pictures!

  <img src="images/BP-intuition-1.png">
- With out input data present we use **forward propagation**
  <img src="images/BP-intuition-2.png">
- The sigmoid function applied to the z values gives the activation values
  - Below we show exactly how the z value is calculated for an example
  <img src="images/BP-intuition-3.png">

### Back propagation

- Back propagation is doing something very similar to forward propagation, but backwards
  - Let's look at the cost function again...
    - Below we have the cost function if there is a single output
    <img src="images/BP-intuition-4.png">
    - This function cycles over each example, so the cost for one example really boils down to this
    <img src="images/BP-intuition-5.png">
    - Which, we can think of as a sigmoidal version of the squared difference
      - So, basically saying, "how well is the network doing on example i "?
    - We can think about a δ term on a unit as the "error" of cost for the activation value associated with a unit
      - More formally, δ is
        <img src="images/BP-intuition-6.png">
        - Where cost is as defined above
        - Cost function is a function of y value and the hypothesis function
    - So, for the output layer, back propagation sets the δ value as [a - y]
      - Difference between activation and actual value
    - We then propagate these values backwards:
      <img src="images/BP-intuition-7.png">
    - Looking at another example to see how we actually calculate the delta value
      <img src="images/BP-intuition-8.png">
    
- So, in effect
  - Back propagation calculates the δ, and those δ values are the weighted sum of the 
    next layer's delta values, weighted by the parameter associated with the links